# Self Driving
## 1. Import

In [15]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## 2. Test Env

In [26]:
env_name = 'CarRacing-v2'
render_mode = None
env = gym.make(env_name,render_mode=render_mode)

In [27]:
env.reset();

In [18]:
env.action_space

Box([-1.  0.  0.], 1.0, (3,), float32)

In [19]:
env.observation_space

Box(0, 255, (96, 96, 3), uint8)

In [25]:
env.close()

In [29]:
episodes =5
for episode in range(1,episodes+1):
    state = env.reset()
    truncated = False
    score = 0
    
    while not truncated:
        if render_mode:
            env.render()
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action) # 'terminated' and 'truncated' instead of 'done'
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:-38.27160493827216
Episode:2 Score:-34.85342019544029
Episode:3 Score:-42.19653179190821
Episode:4 Score:-27.00729927007327
Episode:5 Score:-29.88929889298921


## 3. Train Model

In [40]:
env = gym.make(env_name,render_mode=render_mode)
env = DummyVecEnv([lambda:env])

In [41]:
log_path = os.path.join('Training','Logs')
model = PPO('CnnPolicy',env,verbose=1,tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [42]:
model.learn(total_timesteps=1000)

Logging to Training/Logs/PPO_9
-----------------------------
| time/              |      |
|    fps             | 43   |
|    iterations      | 1    |
|    time_elapsed    | 47   |
|    total_timesteps | 2048 |
-----------------------------


## 4. Save Model

In [43]:
ppo_path =  os.path.join('Training', 'Saved_Models','PPO_Driving_Model')

In [44]:
model.save(ppo_path)

In [45]:
del model

In [46]:
model = PPO.load(ppo_path,env)

Wrapping the env in a VecTransposeImage.


## 5 Evaluate and Test

In [47]:
evaluate_policy(model,env,n_eval_episodes=2,render=True if render_mode else False)

(-112.51004658639431, 0.4566206932067871)

In [48]:
env.close()

In [49]:
episodes =5
for episode in range(1,episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        if render_mode:
            env.render()
        action,_ = model.predict(obs) # outputs (action,state)
        obs, reward, done, info = env.step(action) # Because of the wrapper back to 'done' instead of terminated and truncated
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:[-53.48804]



KeyboardInterrupt

